# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [44]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
rows_before = len(df)
print(df.dtypes)      # Check data types before cleaning 
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

Station Name                    object
Measurement Timestamp           object
Air Temperature                float64
Wet Bulb Temperature           float64
Humidity                         int64
Rain Intensity                 float64
Interval Rain                  float64
Total Rain                     float64
Precipitation Type             float64
Wind Direction                   int64
Wind Speed                     float64
Maximum Wind Speed             float64
Barometric Pressure            float64
Solar Radiation                  int64
Heading                        float64
Battery Life                   float64
Measurement Timestamp Label     object
Measurement ID                  object
dtype: object


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [45]:
# 1. Data Cleaning
print("\n" + "="*80)
print("HANDLING MISSING DATA")
print("="*80)

# Check for missing values
missing_before = df.isnull().sum()
missing_data_info = {}

for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / rows_before) * 100
        missing_data_info[col] = {
            'count': missing_count,
            'percentage': missing_pct
        }
        print(f"{col}: {missing_count} missing ({missing_pct:.1f}%)")

# Handle missing data for numeric columns
print("\nApplying forward-fill for time series data...")
numeric_columns = df.select_dtypes(include=[np.number]).columns
print("Numeric columns to process:", numeric_columns.tolist())

for col in numeric_columns:
    if df[col].isnull().sum() > 0:
        # Forward-fill (appropriate for time series)
        df[col] = df[col].fillna(method='ffill')
        
        # If there are still missing values, use median
        if df[col].isnull().sum() > 0:
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            print(f"  {col}: Forward-filled, then median imputed ({median_value:.2f})")

# Handle missing data for datetime

    if df['Measurement Timestamp'].isnull().sum() > 0:
        df = df.dropna(subset=[col])
        print(f"  {col}: Dropped rows with missing measurement timestamps")

print(f"\nMissing values after handling: {df.isnull().sum().sum()}")

# Outliers detection and removal using IQR method
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    
    # Count outliers
    outlier_info = {}
    
    outliers_lower = (df[col] < lower_bound).sum()
    outliers_upper = (df[col] > upper_bound).sum()
    total_outliers = outliers_lower + outliers_upper
    print(f"{col}: {total_outliers} outliers detected (Lower: {outliers_lower}, Upper: {outliers_upper})")

    if total_outliers > 0:
        print(f"\n{col}:")
        print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
        print(f"  Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Outliers detected: {total_outliers} ({outliers_lower} low, {outliers_upper} high)")
        
    outlier_info[col] = {
            'count': total_outliers,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
    
    # Cap the outliers
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)


#data types validated and converted if necessary
for col in df.columns:
    if col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
print("Data types after conversion:\n", df.dtypes)

# Convert Measurement timestamp to datetime
if 'Measurement Timestamp' in df.columns:
    df['Measurement Timestamp'] = pd.to_datetime(df['Measurement Timestamp'])


#Duplicates removal
duplicates_before = df.duplicated().sum()
print(f"Duplicate rows found: {duplicates_before}")

df = df.drop_duplicates()
rows_after = len(df)

print(f"Rows after removing duplicates: {rows_after}")


# Save cleaned data
df.to_csv("output/q2_cleaned_data.csv", index=False)
rows_after = len(df)



HANDLING MISSING DATA
Air Temperature: 75 missing (0.0%)
Wet Bulb Temperature: 75974 missing (38.7%)
Rain Intensity: 75974 missing (38.7%)
Total Rain: 75974 missing (38.7%)
Precipitation Type: 75974 missing (38.7%)
Barometric Pressure: 146 missing (0.1%)
Heading: 75974 missing (38.7%)

Applying forward-fill for time series data...
Numeric columns to process: ['Air Temperature', 'Wet Bulb Temperature', 'Humidity', 'Rain Intensity', 'Interval Rain', 'Total Rain', 'Precipitation Type', 'Wind Direction', 'Wind Speed', 'Maximum Wind Speed', 'Barometric Pressure', 'Solar Radiation', 'Heading', 'Battery Life']

Missing values after handling: 0
Air Temperature: 97 outliers detected (Lower: 97, Upper: 0)

Air Temperature:
  Q1: 4.30, Q3: 21.50, IQR: 17.20
  Bounds: [-21.50, 47.30]
  Outliers detected: 97 (97 low, 0 high)
Wet Bulb Temperature: 279 outliers detected (Lower: 279, Upper: 0)

Wet Bulb Temperature:
  Q1: 3.20, Q3: 17.60, IQR: 14.40
  Bounds: [-18.40, 39.20]
  Outliers detected: 279 

C:\Users\hp\AppData\Local\Temp\ipykernel_19416\87710976.py:28: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')


Rain Intensity: 6621 outliers detected (Lower: 0, Upper: 6621)

Rain Intensity:
  Q1: 0.00, Q3: 0.00, IQR: 0.00
  Bounds: [0.00, 0.00]
  Outliers detected: 6621 (0 low, 6621 high)
Interval Rain: 15851 outliers detected (Lower: 1, Upper: 15850)

Interval Rain:
  Q1: 0.00, Q3: 0.00, IQR: 0.00
  Bounds: [0.00, 0.00]
  Outliers detected: 15851 (1 low, 15850 high)
Total Rain: 11621 outliers detected (Lower: 0, Upper: 11621)

Total Rain:
  Q1: 11.00, Q3: 198.20, IQR: 187.20
  Bounds: [-269.80, 479.00]
  Outliers detected: 11621 (0 low, 11621 high)
Precipitation Type: 12668 outliers detected (Lower: 0, Upper: 12668)

Precipitation Type:
  Q1: 0.00, Q3: 0.00, IQR: 0.00
  Bounds: [0.00, 0.00]
  Outliers detected: 12668 (0 low, 12668 high)
Wind Direction: 0 outliers detected (Lower: 0, Upper: 0)
Wind Speed: 12221 outliers detected (Lower: 0, Upper: 12221)

Wind Speed:
  Q1: 1.60, Q3: 3.30, IQR: 1.70
  Bounds: [-0.95, 5.85]
  Outliers detected: 12221 (0 low, 12221 high)
Maximum Wind Speed: 4024 o

In [46]:
#2. REPORTING THE ANALYSIS RESULTS

report = []
report.append("DATA CLEANING REPORT")
report.append("=" * 20)
report.append("")
report.append(f"Rows before cleaning: {rows_before}")
report.append("")

# Missing data section
report.append("Missing Data Handling:")
if missing_data_info:
    for col, info in missing_data_info.items():
        report.append(f"- {col}: {info['count']} missing values ({info['percentage']:.1f}%)")
        report.append(f"  Method: Forward-fill (time series)")
        report.append(f"  Result: All missing values filled")
        report.append("")
else:
    report.append("- No missing values detected")
    report.append("")

# Outlier section
report.append("Outlier Handling:")
if outlier_info:
    for col, info in outlier_info.items():
        report.append(f"- {col}: Detected {info['count']} outliers using IQR method (3×IQR)")
        report.append(f"  Method: Capped outliers")
        report.append(f"  Bounds: [{info['lower_bound']:.2f}, {info['upper_bound']:.2f}]")
        report.append(f"  Result: {info['count']} value capped")
        report.append("")
else:
    report.append("- No outliers detected")
    report.append("")

# Duplicates section
report.append(f"Duplicates Removed: {duplicates_before}")
report.append("")



# Save report
report_text = '\n'.join(report)
with open('output/q2_cleaning_report.txt', 'w') as f:
    f.write(report_text)
print("✓ Saved: output/q2_cleaning_report.txt")

✓ Saved: output/q2_cleaning_report.txt


In [47]:
#3. SAVE ROWS AFTER CLEANING
with open('output/q2_rows_cleaned.txt', 'w') as f:
    f.write(str(rows_after))
print("✓ Saved: output/q2_rows_cleaned.txt")

✓ Saved: output/q2_rows_cleaned.txt


In [ ]:
#Decision
# I used ffill for missing numeric data because the dataset is time series based, making it appropriate to carry forward the last known value for any remaining missing.
# we had Air Temperature: 75 missing (0.0%), Wet Bulb Temperature: 75974 missing (38.7%), Rain Intensity: 75974 missing (38.7%), Total Rain: 75974 missing (38.7%)
#Precipitation Type: 75974 missing (38.7%), Barometric Pressure: 146 missing (0.1%), Heading: 75974 missing (38.7%). After ffill, any remaining missing at the start were filled with median.
#At the end, we had 0 missing values. 
# I used the IQR methods to detect outliers. I believe these outliers were errors in the data entry. I chose to cap the outliers them since  we need to preserve the data samples to produce accurate values for time series analysis to produce accurate results.
# Yes, numeric columns were really numeric. However, the measurement timestamp needed to be converted to datetime. 